# Expand the atomic state functions in a LS-coupled basis

In [1]:
using JAC


In atomic spectroscopy, the (standard) LSJ-notation of levels is frequently applied for classifying the low-lying level structures of atoms or ions. This generally requires a *unitary transformation* from a jjJ --> LSJ-coupled basis which critically depend on the shell-structure of the atoms. Since a lack of providing a fast and proper spectroscopic notation from relativistic structure calculations may hamper the interpretation and analysis of many (computed) results, `JAC` readily provides such transformation (though presently restricted to levels with **not more than two open non-relativistic shells**).

Let us first consider the low-lying $1s^2 2s 2p\; ^{1,3}P$ levels of beryllium-like ions, whose LSJ-decompositions quickly changes from a nearly pure LSJ-coupling for neutral beryllium as the nuclear charge increases along the isoelectronic sequence. We can see this, for instance, by considering the $^1P_1 - ^3P_1$ mixing for these ions.

Most easily, the transformation of atomic states from a jjJ-coupled into a LSJ-coupled basis is made after the Hamiltonian matrix has been diagonalized. In particular, such a jjJ --> LSJ transformation is performed if we set `LSjj.Settings(true)` in `Asf.Settings(...)`.

For this, let us first briefly recall the definion of `AsfSettings(..)`

In [2]:
? AsfSettings

search: AsfSettings



`struct  ManyElectron.AsfSettings`       ... a struct for defining the settings for the atomic state functions, i.e. the self-consistent-field (SCF)          and CI computations

```
+ generateScf          ::Bool               ... True, if a SCF need to be generated, and false otherwise (frozen orbitals).
+ breitScf             ::Bool               ... True, if Breit interaction is to be included into the SCF computations.
+ methodScf            ::String             ... Specify the SCF method: ["AL", "OL", "EOL", "meanDFS", "meanHS"].
+ startScf             ::String             ... Specify how the start orbitals are obtained ["hydrogenic", "fromOrbitals"].
+ startOrbitals        ::Dict{Subshell, Orbital}  ... List of start orbitals; hydrogenic orbitals are used 
                                                      if not found in this list.
+ levelsScf            ::Array{Int64,1}     ... Levels on which the optimization need to be carried out.
+ maxIterationsScf     ::Int64              ... maximum number of SCF iterations
+ accuracyScf          ::Float64            ... convergence criterion for the SCF field.
+ shellSequenceScf     ::Array{Subshell,1}  ... Sequence of subshells to be optimized.
+ frozenSubshells      ::Array{Subshell,1}  ... Sequence of subshells to be kept frozen.

+ coulombCI            ::Bool               ... logical flag to include Coulomb interactions.
+ breitCI              ::Bool               ... logical flag to include Breit interactions.
+ qedModel             ::AbstractQedModel   ... model for estimating QED corrections {NoneQed(), QedPetersburg(), QedSydney()}.
+ methodCI             ::String             ... method for diagonalizing the matrix.
+ jjLS                 ::LSjjSettings       ... settings to control a jj-LS transformation of atomic level, if requested.

+ selectLevelsCI       ::Bool                          ... true, if specific level (number)s have been selected.
+ selectedLevelsCI     ::Array{Int64,1}                ... Level number that have been selected.
+ selectSymmetriesCI   ::Bool                          ... true, if specific level symmetries have been selected.
+ selectedSymmetriesCI ::Array{LevelSymmetry,1}        ... Level symmetries that have been selected.
```

---

`ManyElectron.AsfSettings()`  ... constructor for setting the default values.

---

`ManyElectron.AsfSettings(settings::AsfSettings;`

```
        generateScf=..,       breitScf=..,            methodScf=..,          startScf=..,               startOrbitals=..,  
        levelsScf=..,         maxIterationsScf=..,    accuracyScf=..,        shellSequenceScf=..,       frozenSubshells=..,         
        coulombCI=..,         breitCI=..,             qedModel=..,           methodCI=..,               jjLS=..,  
        selectLevelsCI=..,    selectedLevelsCI=..,    selectSymmetriesCI=.., selectedSymmetriesCI=..,   printout::Bool=false)
... constructor for re-defining a settings::AsfSettings.
```


and define it as follows:

In [3]:
asfSettings = AsfSettings(AsfSettings(); generateScf=true, maxIterationsScf=20, jjLS=LSjjSettings(true))

generateScf:          true  
breitScf:             false  
methodScf:            meanDFS  
startScf:             hydrogenic  
startOrbitals:        Dict{Subshell,Orbital}()  
levelsScf:            [1]  
maxIterationsScf:     20  
accuracyScf:          1.0e-6  
shellSequenceScf:     Subshell[]  
frozenSubshells:      Subshell[]  
coulombCI:            true  
breitCI:              false  
qedModel :            NoneQed()  
methodCI:             eigen  
jjLS :                true  
selectLevelsCI:       false  
selectedLevelsCI:     Int64[]  
selectSymmetriesCI:   false  
selectedSymmetriesCI: LevelSymmetry[]  


With these settings, we can calculate the five lowest levels of neutral beryllium due to the $1s^2 (2s^2 + 2s 2p)$ configurations, and which includes the $^1P_1 - ^3P_1$ levels.

In [4]:
wa = Atomic.Computation(Atomic.Computation(), name="jj-LS level transformation", 
                        grid=JAC.Radial.Grid(true), nuclearModel=Nuclear.Model(4.); 
                        configs=[Configuration("[He] 2s^2"), Configuration("[He] 2s 2p")],  
                        asfSettings=asfSettings )
perform(wa)


... in performSCF ...
>> include Configuration: 1s_1/2^2 2s_1/2^2 
>> include Configuration: 1s_1/2^2 2s_1/2^1 2p_1/2^0 2p_3/2^1 
>> include Configuration: 1s_1/2^2 2s_1/2^1 2p_1/2^1 2p_3/2^0 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
>> (Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 4.0 with mass = 8.08, radius R = 2.247554858082373 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -8.00170451e+00    -8.00170477e+00    +3.27226863e-08    
      2      2s_1/2      -2.00053329e+00    -2.00053275e+00    -2.70611614e-07    
      3      3s_1/2   


Apart from the successfull SCF iteration, the final part of this output shows the LS-expansion of all selected levels. While, in these simple computations, the $2s^2\; ^1S_0$ and the $2s 2p\; ^3P_{0,2}$ must be pure, configuation mixing will generally affect the two $2s 2p\; ^{1,3}P_1$ already in this single-configurations approximation. For neutral beryllium, however, this mixing is so *small* that both these levels appear basically *pure* in LSJ notation.

We can repeat analogue computations for beryllium-like argon ($Z=18$) and xenon ($Z=54$), with focus especially to the corresponding LSJ-expansions:

In [5]:
wa = Atomic.Computation(wa, nuclearModel=Nuclear.Model(18.))
perform(wa)


... in performSCF ...
>> include Configuration: 1s_1/2^2 2s_1/2^2 
>> include Configuration: 1s_1/2^2 2s_1/2^1 2p_1/2^0 2p_3/2^1 
>> include Configuration: 1s_1/2^2 2s_1/2^1 2p_1/2^1 2p_3/2^0 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
>> (Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 18.0 with mass = 37.62, radius R = 3.37121133160794 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -1.62704538e+02    -1.62704858e+02    +1.97039606e-06    
      2      2s_1/2      -4.07203374e+01    -4.07203638e+01    +6.49689020e-07    
      3      3s_1/2  

In [6]:
wa = Atomic.Computation(wa, nuclearModel=Nuclear.Model(54.))
perform(wa)


... in performSCF ...
>> include Configuration: 1s_1/2^2 2s_1/2^2 
>> include Configuration: 1s_1/2^2 2s_1/2^1 2p_1/2^0 2p_3/2^1 
>> include Configuration: 1s_1/2^2 2s_1/2^1 2p_1/2^1 2p_3/2^0 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
>> (Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 54.0 with mass = 122.58, radius R = 4.722849091608491 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -1.51935928e+03    -1.51947334e+03    +7.50765231e-05    
      2      2s_1/2      -3.83773854e+02    -3.83790162e+02    +4.24935905e-05    
      3      3s_1/2

From the last two computations, we see that relative contribution of the $^3P_1$ (non-relativistic LSJ) configuration state of the $2s 2p\; ^{3}P_1$ level drops from 1.00 ($Z=4$) to 0.85 ($Z=18$) to 0.76 ($Z=54$), and similarly also for the $2s 2p\; ^{1}P_1$ level. This change in non-relativistic LSJ decomposition is also the main reason why the $2s 2p\; ^{3}P_1 - 2s^2\; ^1S_0$ (intercombination) line is very week, when compared with the $2s 2p\; ^{1}P_1 - 2s^2\; ^1S_0$ resonance line, for neutral beryllium but become comparable to each other for medium and heavy elements. The transition strength of all these transitions arise from (resonance) decay within the singlett system, while the $2s 2p\; ^{3}P$ represent the lowest term (levels) of the triplett system.

The jjJ --> LSJ transformation is presently restricted to configuration expansions for which all non-relativistic CSF have **not more than two open non-relativistic shells**. This restriction can be released if the re-coupling coefficients are implemented for a larger number of open shells. Since these jjJ --> LSJ transformation can become quite expensive for complex shell structure, they can be further controlled by selecting individual levels (or symmetries of levels) as well as a minimum with which a (relativistic) CSF must contribute to least on the (selected) levels to be taken into account in the expansion.

In `JAC`, an improved control about such jjJ --> LSJ transformation is prepared (but not yet realized in all detail) by the parameters of the `LSjjSettings`, and is shown together with its  default settings:

In [7]:
?LSjjSettings

search: LSjjSettings



`struct  ManyElectron.LSjjSettings`  ... defines a type for the details and parameters for performing jj-LS expansions.

```
+ makeIt         ::Bool            ... True, if the jj-LS expansion is to be made and false otherwise.
+ minWeight      ::Float64         ... minimum weight with which a (relativistic) CSF must contribute to (at least one) level.
+ printWeight    ::Float64         ... minimum weight of a nonrelativistic CSF to be printed out in the final expansion.
+ selectLevels   ::Bool            ... True, if levels are selected individually for the computations.
+ selectedLevels ::Array{Int64,1}  ... List of levels.
```

---

`LSjjSettings(makeIt::Bool)`  ... constructor for the default values of jj-LS transformations.


In [8]:
LSjjSettings(true)

makeIt:            true  
minWeight:         0.05  
printWeight:       0.1  
selectLines:       false  
selectedLines:     Int64[]  



As seen from beryllium-example above, the LS-coupling notation of levels can often be derived for simple shell structure by considering the fine-structure splitting (energies) and the total $J$ values of the atomic levels. For beryllium-like ions, for exmaple, the $^3P_{0,1,2}$ triplet can be identified quite easily by just looking for it fine-structure energies as obtained from a simple CI computations.

This identification becomes much less obvious if open $d$-shells or even $f$-shells are involved
as, for example, for the levels of $3d^3$ ground configuration of scandium-like iron. Following similar lines as above, we obtain: 


In [9]:
wa = Atomic.Computation(Atomic.Computation(), name="jj-LS level transformation", 
                        grid=JAC.Radial.Grid(true), nuclearModel=Nuclear.Model(26.); 
                        configs=[Configuration("[Ar] 3d^3")], asfSettings=asfSettings )
perform(wa)


... in performSCF ...
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^2 3p_3/2^4 3d_3/2^0 3d_5/2^3 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^2 3p_3/2^4 3d_3/2^1 3d_5/2^2 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^2 3p_3/2^4 3d_3/2^2 3d_5/2^1 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^2 3p_3/2^4 3d_3/2^3 3d_5/2^0 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
>> (Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 26.0 with mass = 55.38, radius R = 3.756573410087755 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  ---


Apparently, there are 19 levels which belong to the single $3d^3$ configuration; although there is usually one dominant CSF in non-relativistic LSJ -coupling, the mixing of different terms can become quite sizeable (see levels 14 and 15).
How many levels do we see ... and what can be said about the LS-coupling of the levels in the ground configuration ?? 

Of course, the complexity increases rapidly if we consider the level structure of the potassium-like iron but with a $2p$ hole:

In [10]:
wa = Atomic.Computation(wa, configs=[Configuration("[Ne] 3s^2 3p^5 3d^2")])
perform(wa)


... in performSCF ...
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^1 3p_3/2^4 3d_3/2^0 3d_5/2^2 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^1 3p_3/2^4 3d_3/2^1 3d_5/2^1 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^1 3p_3/2^4 3d_3/2^2 3d_5/2^0 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^2 3p_3/2^3 3d_3/2^0 3d_5/2^2 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^2 3p_3/2^3 3d_3/2^1 3d_5/2^1 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^2 3p_3/2^3 3d_3/2^2 3d_5/2^0 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
>> (Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 26.0 with mass = 55.38, radius R = 3.756573410087755 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 



This results in 45 fine-structure levels, and which are shown together with their corresponding LSJ expansions. For various applications, it migt be desirable to better recognized the parents terms in this LSJ expansion, a feature that could be implemented quite easily.
